In [9]:
import urllib.request as req
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

import pymysql

In [328]:
url='https://movie.naver.com/movie/running/current.naver'
res = requests.get(url)
res.encoding='utf-8'

bs = BeautifulSoup(res.text, 'html.parser')
movie_list = bs.select('div.lst_wrap > ul > li')

base='https://movie.naver.com'

pd_list =[]

for movie in movie_list:
    img_url = movie.select_one('.thumb>a>img')['src']
    rating = movie.select_one('.tit>span').string
    title = movie.select_one('.tit>a').string
    detail_url=movie.select_one('.tit>a')['href']
    genres,time,date= movie.select_one('dl > dd:nth-child(3) > dl > dd').text.replace('\t','').replace('\n','').replace('\r','').replace('분','').replace(' 개봉','').split('|')
    genres = genres.split(', ')
    director = movie.select_one('dl > dd:nth-child(3) > dl > dd:nth-child(4)>span>a').string
    actors= movie.select_one('dl > dd:nth-child(3) > dl > dd:nth-child(6)>span')
    actors = actors.text.replace('\t','').replace('\n','').replace('\r','').split(', ') if actors else []
    user_star, *writer_star = [num.text for num in movie.select('.star .num')]
    writer_star=''.join(writer_star)
    
    temp, story = get_detail(base+detail_url)
    
    
    pd_list.append((title,rating,genres,time,date, director, actors,user_star,writer_star,temp,story))
    
df = pd.DataFrame(pd_list,columns=['title','rating','genres','time','date','director','actors','user_star','writer_star','temp','story'])
df

,title,rating,genres,time,date,director,actors,user_star,writer_star,temp,story
0,올리 마키의 가장 행복한 날,12세 관람가,"[멜로/로맨스, 드라마]",93,2022.06.02,유호 쿠오스마넨,"[에로 밀로노프, 자코 라티, 우나 아이롤라]",0.00,8.00,[정보 없음],평범한 제빵사에서 권투 영웅으로 떠오른 ‘올리’전 국민이 고대하는 세계 챔피언십을 ...
1,"괴물, 유령, 자유인",15세 관람가,"[SF, 멜로/로맨스]",77,2022.06.02,홍지영,"[엄태나, 윤해신, 권기하]",0.00,,[정보 없음],성심과 은수는 동성연인이다.사회에서는 결코 인정하지 않기에결국 서로에 대한 믿음은 ...
2,쥬라기 월드: 도미니언,12세 관람가,"[액션, 모험]",147,2022.06.01,콜린 트레보로우,"[크리스 프랫, 브라이스 달라스 하워드]",6.36,,"[3%10, 30%20, 44%30, 17%40, 6%50]","공룡들의 터전이었던 이슬라 누블라 섬이 파괴된 후,마침내 공룡들은 섬을 벗어나 세상..."
3,극장판 포켓몬스터DP: 기라티나와 하늘의 꽃다발 쉐이미,전체 관람가,[애니메이션],96,2022.06.01,유야마 쿠니히코,"[이선호, 김영선, 마츠모토 리카, 오오타니 이쿠에, 우에다 유지]",7.45,6.00,"[20%10, 0%20, 40%30, 40%40, 0%50]",끝나지 않은 전설의 포켓몬들의 배틀로위험에 빠진 반전 세계와 현실 세계를 구하기 위...
4,카시오페아,12세 관람가,[드라마],102,2022.06.01,신연식,"[안성기, 서현진, 주예림]",9.37,6.50,"[14%10, 29%20, 14%30, 14%40, 29%50]","이혼 후 변호사, 엄마로 완벽한 삶을 살아가려고 노력하는 수진은 하나뿐인 딸 지나의..."
...,...,...,...,...,...,...,...,...,...,...,...
79,여름 이야기,12세 관람가,"[코미디, 멜로/로맨스]",114,1998.06.13,에릭 로메르,"[멜빌 푸포, 아만다 랑글렛]",8.12,6.00,"[10%10, 26%20, 58%30, 3%40, 3%50]",가스파르(Gaspard: 멜빌 포포드 분)는 여름날 혼자 해변에 간다. 여자 친구 ...
80,로맨스 빠빠,15세 관람가,"[가족, 멜로/로맨스]",131,1960.01.28,신상옥,"[최은희, 김진규, 강신성일, 김승호, 주증녀]",8.52,,[정보 없음],보험회사 사원인 그는 2남3녀의 아버지이다. 보험회사에 감원 선풍이 인다. 그는 건...
81,OK 목장의 결투,12세 관람가,"[서부, 액션, 스릴러]",117,1958.01.01,존 스터지스,"[버트 랭커스터, 커크 더글러스]",8.68,,[정보 없음],전직 치과의사였던 닥 할리데이(Dr. John 'Doc' Holliday: 커크 더...
82,젊은이의 양지,15세 관람가,"[드라마, 느와르, 멜로/로맨스]",122,1953.11.26,조지 스티븐스,"[몽고메리 클리프트, 엘리자베스 테일러, 쉘리 윈터스]",8.81,,[정보 없음],조지 이스트맨(몽고메리 클리프트 분)은 가난하지만 잘 생기고 매력적이며 야망에 찬 ...


In [326]:
def get_detail(url):
    res = requests.get(url)
    res.encoding='utf-8'
    bs=BeautifulSoup(res.text,'html.parser')

    mstory = ''.join( txt.replace('\r','').replace('\xa0','') for txt in bs.select_one('.con_tx').find_all(text=True))
    
    graph = bs.select_one('div.mv_info div.bar_graph')
    
    temp=[]
    
    if graph:
        pers = graph.select('.graph_percent')
        ages = graph.select('.graph_legend>span')
        
       
        for per,age in zip(pers,ages):
            temp.append(per.text+age.text)
    else:
        temp.append('정보 없음')
    
    return (temp,mstory)
    
    
detail_url='https://movie.naver.com/movie/bi/mi/basic.naver?code=191646'
temp, story = get_detail(detail_url)
print(temp,story)

['3%10', '30%20', '44%30', '17%40', '6%50'] 공룡들의 터전이었던 이슬라 누블라 섬이 파괴된 후,마침내 공룡들은 섬을 벗어나 세상 밖으로 출몰한다. 지상에 함께 존재해선 안 될 위협적 생명체인 공룡의 등장으로인류 역사상 겪어보지 못한 사상 최악의 위기를 맞이한 인간들.지구의 최상위 포식자 자리를 걸고 인간과 공룡의 최후의 사투가 펼쳐진다.


In [277]:
detail_url='https://movie.naver.com/movie/bi/mi/basic.naver?code=191646'
get_detail(detail_url)

이제 모든 것이 끝난다,
지상 최대 블록버스터의 압도적 피날레!
공룡들의 터전이었던 이슬라 누블라 섬이 파괴된 후,마침내 공룡들은 섬을 벗어나 세상 밖으로 출몰한다. 지상에 함께 존재해선 안 될 위협적 생명체인 공룡의 등장으로인류 역사상 겪어보지 못한 사상 최악의 위기를 맞이한 인간들.지구의 최상위 포식자 자리를 걸고 인간과 공룡의 최후의 사투가 펼쳐진다.
